In [8]:
import torch
import time
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    pipeline
)
import evaluate
from fastapi import FastAPI
from google.colab import files


In [21]:
df = pd.read_csv('data2.csv')

In [23]:
# Load data
data = [{"text": f"Input: {i} → Output: {o}"} for i, o in zip(df['Input'], df['Output'])]
dataset = Dataset.from_list(data)
# Tokenizer and model
model_name = "distilgpt2"
token = "HUGGINGFACE_TOKEN"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
# Add the padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token  # or '[PAD]' if you prefer
model = AutoModelForCausalLM.from_pretrained(model_name, token=token)


def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/4342 [00:00<?, ? examples/s]

In [24]:
training_args = TrainingArguments(
    output_dir="./model_output",
    per_device_train_batch_size=4,  # adjust for your GPU memory
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    evaluation_strategy="no",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)
if torch.cuda.is_available():
    model.to('cuda')
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-6b8cc57bed4f>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,2.407900
20,0.587600
30,0.365700
40,0.271500
50,0.229200
60,0.207300
70,0.188800
80,0.206300
90,0.176300
100,0.179200


TrainOutput(global_step=3258, training_loss=0.1439920323549274, metrics={'train_runtime': 598.9205, 'train_samples_per_second': 21.749, 'train_steps_per_second': 5.44, 'total_flos': 425456434151424.0, 'train_loss': 0.1439920323549274, 'epoch': 3.0})

In [25]:
model.save_pretrained("fantasy_item_model2")
tokenizer.save_pretrained("fantasy_item_model2")

('fantasy_item_model2/tokenizer_config.json',
 'fantasy_item_model2/special_tokens_map.json',
 'fantasy_item_model2/vocab.json',
 'fantasy_item_model2/merges.txt',
 'fantasy_item_model2/added_tokens.json',
 'fantasy_item_model2/tokenizer.json')

In [27]:
!zip -r fantasy_item_model2.zip fantasy_item_model2 # Change the filename here to match the saved model directory and the download filename
from google.colab import files
files.download("fantasy_item_model2.zip")

  adding: fantasy_item_model2/ (stored 0%)
  adding: fantasy_item_model2/special_tokens_map.json (deflated 60%)
  adding: fantasy_item_model2/vocab.json (deflated 59%)
  adding: fantasy_item_model2/merges.txt (deflated 53%)
  adding: fantasy_item_model2/generation_config.json (deflated 24%)
  adding: fantasy_item_model2/tokenizer_config.json (deflated 54%)
  adding: fantasy_item_model2/tokenizer.json (deflated 82%)
  adding: fantasy_item_model2/config.json (deflated 52%)
  adding: fantasy_item_model2/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_path = "fantasy_item_model2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Input: Wand, ice element → Output:"
result = generator(prompt, max_length=30, do_sample=True, top_k=50, top_p=0.95)
print(result[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Input: Wand, ice element → Output: GlacierShard of Mire


In [29]:
import evaluate

rouge = evaluate.load("rouge")

predictions = []
references = []

for idx, row in df.iterrows():
    prompt = f"Input: {row['Input']} → Output:"
    output = generator(prompt, max_length=30, do_sample=False)[0]['generated_text']
    predicted = output.replace(prompt, "").strip()

    predictions.append(predicted)
    references.append(row['Output'].strip())

results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': np.float64(0.26967694647335455), 'rouge2': np.float64(0.05500811563685816), 'rougeL': np.float64(0.2587493511146223), 'rougeLsum': np.float64(0.2587649791991129)}


In [24]:
%%writefile app.py
from fastapi import FastAPI
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

app = FastAPI()
start_time = time.time()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained("fantasy_item_model2").to(device)
tokenizer = AutoTokenizer.from_pretrained("fantasy_item_model2")

@app.get("/generate")
def generate(prompt: str):
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=20, do_sample=True, top_k=50, temperature=0.8)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return {"output": result}
    except Exception as e:
        return {"error": str(e)}

@app.get("/status")
def status():
    uptime = time.time() - start_time
    return {
        "status": "running",
        "model": "fantasy_item_model2",
        "device": str(device),
        "uptime": f"{uptime:.2f} seconds"
    }


Overwriting app.py


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("shuvankar/fantasy-item-model2")
tokenizer = AutoTokenizer.from_pretrained("shuvankar/fantasy-item-model2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]